<a href="https://colab.research.google.com/github/nanafish/ORS/blob/main/%E3%82%B7%E3%83%B3%E3%83%97%E3%83%AB%E3%82%A2%E3%82%A4%E3%82%BD%E3%82%AF%E3%83%AD%E3%83%B3%E2%86%92%E4%BA%BA%E5%8F%A3%E3%83%AA%E3%82%B9%E3%83%88%EF%BC%8B%E3%83%A9%E3%83%99%E3%83%AB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# ORS Isochrone 必ず30/45/60を出力する版（1セル）
# - 返却値が欠けても最も近いポリゴンをコピーして埋める
# - まとめKML + 個別KML + リングKML
# 使用ファイルN03-20240101.geojson と【総計】市区町村別年齢階級別人口(2025.8).xlsx
# ============================================

!pip -q install openrouteservice shapely simplekml

import os, json, datetime, shutil, math
import openrouteservice as ors
from shapely.geometry import shape, Polygon, MultiPolygon, mapping
from shapely.ops import unary_union
import simplekml
from google.colab import files

# ---- 入力 ----
ORS_API_KEY = input("ORS_API_KEY: ").strip()
VENUE_NAME  = input("会場名: ").strip()
coord_raw   = input("中心座標（緯度, 経度）: ").strip()

TARGET_MIN  = [30,45,60]
PROFILE     = "driving-car"
OUTDIR      = f"{VENUE_NAME}_iso_{PROFILE}_" + datetime.datetime.now().strftime("%Y%m%d_%H%M")
os.makedirs(OUTDIR, exist_ok=True)

# ---- 座標 ----
def latlon_to_lonlat(s):
    lat_str, lon_str = [x.strip() for x in s.split(",")]
    return [float(lon_str), float(lat_str)], float(lat_str)

center_lonlat, lat0 = latlon_to_lonlat(coord_raw)

# ---- ORS呼び出し ----
client = ors.Client(key=ORS_API_KEY)
ranges_sec = [m*60 for m in TARGET_MIN]
resp = client.isochrones(
    locations=[center_lonlat],
    profile=PROFILE,
    range=ranges_sec,
    range_type="time",
    units="m",
    smoothing=0.0
)
features = resp["features"]

# ---- 補助関数 ----
def to_multipolygon(geom):
    g = shape(geom)
    if isinstance(g, Polygon):      return MultiPolygon([g])
    if isinstance(g, MultiPolygon): return g
    return g.buffer(0)

def save_kml_polygon(kml_obj, geom, name_text, desc_text):
    def add_poly(p):
        poly = kml_obj.newpolygon(
            name=name_text,
            description=desc_text,
            outerboundaryis=[(x,y) for (x,y) in p.exterior.coords]
        )
        if p.interiors:
            poly.innerboundaryis = [[(x,y) for (x,y) in r.coords] for r in p.interiors]
    if isinstance(geom, Polygon): add_poly(geom)
    elif isinstance(geom, MultiPolygon):
        for p in geom.geoms: add_poly(p)
    rp = geom.representative_point()
    kml_obj.newpoint(name=name_text, description=desc_text, coords=[(rp.x, rp.y)])

# ---- 返却結果を整理 ----
actuals = {int(f["properties"]["value"]/60): to_multipolygon(f["geometry"]) for f in features}

# ---- バケットごとに必ず用意 ----
assign = {}
for target in TARGET_MIN:
    if target in actuals:
        assign[target] = {"geom": actuals[target], "ors_value_min": target}
    else:
        # 一番近い返却ポリゴンをコピー
        nearest = min(actuals.keys(), key=lambda m: abs(m-target))
        assign[target] = {"geom": actuals[nearest], "ors_value_min": nearest}

# ---- まとめKML ----
kml_all = simplekml.Kml()
for m in TARGET_MIN:
    g = assign[m]["geom"]
    ors_val = assign[m]["ors_value_min"]
    desc = f"Named={m}min / ORS返却≈{ors_val}min"
    save_kml_polygon(kml_all, g, f"{VENUE_NAME} {m}min", desc)
kml_all.save(os.path.join(OUTDIR, f"{VENUE_NAME}_iso_30_45_60.kml"))

# ---- 個別KML ----
for m in TARGET_MIN:
    g = assign[m]["geom"]; ors_val = assign[m]["ors_value_min"]
    base = f"{VENUE_NAME}_iso_{m}min"
    desc = f"Named={m}min / ORS返却≈{ors_val}min"
    kml = simplekml.Kml(); save_kml_polygon(kml, g, f"{VENUE_NAME} {m}min", desc)
    kml.save(os.path.join(OUTDIR, base + ".kml"))

# ---- リング（非重複）----
rings = []
union_prev = None
for m in TARGET_MIN:
    g = assign[m]["geom"]
    if union_prev is None:
        ring = g; frm = 0
    else:
        ring = unary_union(g).difference(union_prev).buffer(0); frm = prev
        union_prev = unary_union([union_prev, g])
    rings.append((frm,m,ring))
    prev = m

for frm,m,ring in rings:
    base = f"{VENUE_NAME}_ring_{frm}_{m}min"
    kml = simplekml.Kml(); save_kml_polygon(kml, ring, base, f"ring {frm}-{m} min")
    kml.save(os.path.join(OUTDIR, base + ".kml"))

# ---- ZIP ----
zip_path = f"{OUTDIR}.zip"
shutil.make_archive(OUTDIR, "zip", OUTDIR)
files.download(zip_path)

print("=== 完了 ===")
print("必ず30/45/60を出力しました →", OUTDIR)


ORS_API_KEY: eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6IjRjM2JjNmVjNGNkMTRhYTU4ZWEyZmFmM2QxM2I0ZjJhIiwiaCI6Im11cm11cjY0In0=
会場名: イオン弘前樋の口
中心座標（緯度, 経度）: 40.60328677690059, 140.44436913427185


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

=== 完了 ===
必ず30/45/60を出力しました → イオン弘前樋の口_iso_driving-car_20251119_0637


In [1]:
# ============================================
# ORS Isochrone 必ず30/45/60を出力する版（1セル・家マーク付き）
# - 返却値が欠けても最も近いポリゴンをコピーして埋める
# - まとめKML + 個別KML + リングKML
# - 各KMLに「入力座標の家マーク」ピンを1つだけ追加
# 使用ファイル N03-20240101.geojson と【総計】市区町村別年齢階級別人口(2025.8).xlsx
# ============================================

!pip -q install openrouteservice shapely simplekml

import os, json, datetime, shutil, math
import openrouteservice as ors
from shapely.geometry import shape, Polygon, MultiPolygon, mapping
from shapely.ops import unary_union
import simplekml
from google.colab import files

# ---- 入力 ----
ORS_API_KEY = input("ORS_API_KEY: ").strip()
VENUE_NAME  = input("会場名: ").strip()
coord_raw   = input("中心座標（緯度, 経度）: ").strip()

TARGET_MIN  = [30, 45, 60]
PROFILE     = "driving-car"
OUTDIR      = f"{VENUE_NAME}_iso_{PROFILE}_" + datetime.datetime.now().strftime("%Y%m%d_%H%M")
os.makedirs(OUTDIR, exist_ok=True)

# ---- 座標 ----
def latlon_to_lonlat(s):
    lat_str, lon_str = [x.strip() for x in s.split(",")]
    return [float(lon_str), float(lat_str)], float(lat_str)

center_lonlat, lat0 = latlon_to_lonlat(coord_raw)  # [lon, lat]

# ---- ORS呼び出し ----
client = ors.Client(key=ORS_API_KEY)
ranges_sec = [m*60 for m in TARGET_MIN]
resp = client.isochrones(
    locations=[center_lonlat],
    profile=PROFILE,
    range=ranges_sec,
    range_type="time",
    units="m",
    smoothing=0.0
)
features = resp["features"]

# ---- 補助関数 ----
def to_multipolygon(geom):
    g = shape(geom)
    if isinstance(g, Polygon):
        return MultiPolygon([g])
    if isinstance(g, MultiPolygon):
        return g
    return g.buffer(0)

def save_kml_polygon(kml_obj, geom, name_text, desc_text):
    """ポリゴンのみ書き出し（代表点のピンは作らない）"""
    def add_poly(p):
        poly = kml_obj.newpolygon(
            name=name_text,
            description=desc_text,
            outerboundaryis=[(x, y) for (x, y) in p.exterior.coords]
        )
        if p.interiors:
            poly.innerboundaryis = [[(x, y) for (x, y) in r.coords] for r in p.interiors]

    if isinstance(geom, Polygon):
        add_poly(geom)
    elif isinstance(geom, MultiPolygon):
        for p in geom.geoms:
            add_poly(p)

def add_home_pin(kml_obj, center_lonlat, name_text):
    """入力座標に家マークのピンを1つ追加"""
    pnt = kml_obj.newpoint(
        name=f"{name_text}（会場）",
        description="中心座標",
        coords=[tuple(center_lonlat)]  # (lon, lat)
    )
    # 家マークアイコン（Googleの標準KMLアイコン）
    pnt.style.iconstyle.icon.href = "http://maps.google.com/mapfiles/kml/shapes/homegardenbusiness.png"
    pnt.style.iconstyle.scale = 1.2

# ---- 返却結果を整理 ----
actuals = {int(f["properties"]["value"] / 60): to_multipolygon(f["geometry"]) for f in features}

# ---- バケットごとに必ず用意 ----
assign = {}
for target in TARGET_MIN:
    if target in actuals:
        assign[target] = {"geom": actuals[target], "ors_value_min": target}
    else:
        # 一番近い返却ポリゴンをコピー
        nearest = min(actuals.keys(), key=lambda m: abs(m - target))
        assign[target] = {"geom": actuals[nearest], "ors_value_min": nearest}

# ---- まとめKML ----
kml_all = simplekml.Kml()
for m in TARGET_MIN:
    g = assign[m]["geom"]
    ors_val = assign[m]["ors_value_min"]
    desc = f"Named={m}min / ORS返却≈{ors_val}min"
    save_kml_polygon(kml_all, g, f"{VENUE_NAME} {m}min", desc)

# 家マークピン（まとめKML）
add_home_pin(kml_all, center_lonlat, VENUE_NAME)
kml_all.save(os.path.join(OUTDIR, f"{VENUE_NAME}_iso_30_45_60.kml"))

# ---- 個別KML ----
for m in TARGET_MIN:
    g = assign[m]["geom"]
    ors_val = assign[m]["ors_value_min"]
    base = f"{VENUE_NAME}_iso_{m}min"
    desc = f"Named={m}min / ORS返却≈{ors_val}min"
    kml = simplekml.Kml()
    save_kml_polygon(kml, g, f"{VENUE_NAME} {m}min", desc)
    # 家マークピン（個別KML）
    add_home_pin(kml, center_lonlat, VENUE_NAME)
    kml.save(os.path.join(OUTDIR, base + ".kml"))

# ---- リング（非重複）----
rings = []
union_prev = None
prev_m = None

for i, m in enumerate(TARGET_MIN):
    g = assign[m]["geom"]
    if i == 0:
        ring = g
        frm = 0
        union_prev = g  # ★ 最初のポリゴンを保存
    else:
        # ひとつ前までの union を引いてリング化
        ring = g.difference(union_prev).buffer(0)
        frm = prev_m
        union_prev = unary_union([union_prev, g])
    rings.append((frm, m, ring))
    prev_m = m

for frm, m, ring in rings:
    base = f"{VENUE_NAME}_ring_{frm}_{m}min"
    kml = simplekml.Kml()
    save_kml_polygon(kml, ring, base, f"ring {frm}-{m} min")
    # 家マークピン（リングKML）
    add_home_pin(kml, center_lonlat, VENUE_NAME)
    kml.save(os.path.join(OUTDIR, base + ".kml"))

# ---- ZIP ----
zip_path = f"{OUTDIR}.zip"
shutil.make_archive(OUTDIR, "zip", OUTDIR)
files.download(zip_path)

print("=== 完了 ===")
print("必ず30/45/60を出力しました →", OUTDIR)
print("各KMLに中心座標の家マークピンを1つだけ追加しています。")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
ORS_API_KEY: eyJvcmciOiI1YjNjZTM1OTc4NTExMTAwMDFjZjYyNDgiLCJpZCI6IjRjM2JjNmVjNGNkMTRhYTU4ZWEyZmFmM2QxM2I0ZjJhIiwiaCI6Im11cm11cjY0In0=
会場名: イオンモール新利府
中心座標（緯度, 経度）: 38.327618799660684, 140.97197055722862


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

=== 完了 ===
必ず30/45/60を出力しました → イオンモール新利府_iso_driving-car_20251126_2335
各KMLに中心座標の家マークピンを1つだけ追加しています。


In [ ]:
# === 汎用版：KML圏域 × 国交省N03行政界 × e-Stat人口 ===
!pip install -q shapely pandas openpyxl
import pandas as pd, re, json, glob, xml.etree.ElementTree as ET
from shapely.geometry import shape, Polygon
from shapely.ops import unary_union
from google.colab import files

# ---------- 共通ユーティリティ ----------
def norm_name(s):
    if pd.isna(s): return ""
    s = str(s).strip().replace("　","").replace(" ","")
    s = re.sub(r"（.*?）|\(.*?\)", "", s)
    s = re.sub(r".+郡", "", s)
    if "区" in s and "市" in s: s = re.sub(r".+市", "", s)
    return s

def parse_kml_union(path):
    ns = {"kml":"http://www.opengis.net/kml/2.2"}
    root = ET.parse(path).getroot()
    coords = root.findall(".//kml:Polygon//kml:outerBoundaryIs//kml:LinearRing//kml:coordinates", ns)
    polys=[]
    for c in coords:
        pts=[]
        for tok in re.split(r"\s+", (c.text or "").strip()):
            parts = tok.split(",")
            if len(parts)>=2:
                try: pts.append((float(parts[0]), float(parts[1])))
                except: pass
        if len(pts)>=3:
            p=Polygon(pts)
            if p.is_valid and p.area>0: polys.append(p)
    return unary_union(polys) if len(polys)>1 else polys[0]

def pick_munis(union, gj, thr=0.3):
    rows=[]
    for ft in gj.get("features", []):
        g=ft.get("geometry")
        if not g: continue
        shp=shape(g)
        inter=shp.intersection(union)
        if inter.is_empty: continue
        ratio=inter.area/shp.area if shp.area>0 else 0
        if ratio>=thr:
            pref=ft["properties"].get("N03_001","")
            muni=ft["properties"].get("N03_005") or ft["properties"].get("N03_004") or ft["properties"].get("N03_003") or ft["properties"].get("N03_007") or ""
            c=shp.centroid
            rows.append({"都道府県":pref,"市区町村":muni,"lat":c.y,"lng":c.x})
    return pd.DataFrame(rows).drop_duplicates(subset=["都道府県","市区町村"]).reset_index(drop=True)

def load_pop():
    xlsx = sorted(glob.glob("*年齢階級別人口*2025.8*.xlsx"))
    if not xlsx: raise FileNotFoundError("人口Excelが見つかりません。")
    df_raw = pd.read_excel(xlsx[0], sheet_name="年齢別人口（市区町村別）【総計】", header=None, dtype=str)
    fixed=["団体コード","都道府県名","市区町村名","性別"]
    ages=df_raw.iloc[1,4:].tolist()
    cols=fixed+ages
    df=df_raw.iloc[3:,:len(cols)].copy(); df.columns=cols
    df=df.rename(columns={"都道府県名":"都道府県","市区町村名":"市区町村"})
    for a in ["25歳～29歳","30歳～34歳","35歳～39歳","40歳～44歳"]:
        df[a]=pd.to_numeric(df[a].astype(str).str.replace(",",""),errors="coerce")
    male=df[df["性別"].astype(str).str.strip()=="男"].copy()
    male["25～44歳男性"]=male["25歳～29歳"]+male["30歳～34歳"]+male["35歳～39歳"]+male["40歳～44歳"]
    male["都道府県_norm"]=male["都道府県"].astype(str).str.strip()
    male["市区町村_norm"]=male["市区町村"].map(norm_name)
    male["key"]=male["都道府県_norm"]+" / "+male["市区町村_norm"]
    return male[["key","25～44歳男性"]]

def attach_and_export(df_pts,pop,prefix):
    # 重複削除
    df_pts=df_pts.drop_duplicates(subset=["都道府県","市区町村"]).reset_index(drop=True)
    df_pts["都道府県_norm"]=df_pts["都道府県"].astype(str).str.strip()
    df_pts["市区町村_norm"]=df_pts["市区町村"].map(norm_name)
    df_pts["key"]=df_pts["都道府県_norm"]+" / "+df_pts["市区町村_norm"]
    res=df_pts.merge(pop,on="key",how="left")[["都道府県","市区町村","lat","lng","25～44歳男性"]]
    # 平均座標まとめ
    res=res.groupby(["都道府県","市区町村"],as_index=False).agg({"lat":"mean","lng":"mean","25～44歳男性":"first"})
    # CSV出力
    lst=res[["都道府県","市区町村","25～44歳男性"]].sort_values(["都道府県","市区町村"]).reset_index(drop=True)
    total=pd.DataFrame({"都道府県":["(合計)"],"市区町村":[f"対象自治体数:{len(lst)}"],"25～44歳男性":[lst["25～44歳男性"].sum(skipna=True)]})
    out=pd.concat([lst,total],ignore_index=True)
    pop_csv=f"{prefix}_25-44male_population.csv"; out.to_csv(pop_csv,index=False,encoding="utf-8-sig")
    res["Population"]=pd.to_numeric(res["25～44歳男性"],errors="coerce").fillna(0).astype(int)
    res["Label"]=res["市区町村"]+"（"+res["Population"].map(lambda x:f"{x:,}人")+"）"
    label_csv=f"{prefix}_25-44male_LabelMap.csv"
    res.rename(columns={"市区町村":"Name"})[["Name","Label","lat","lng","Population"]].to_csv(label_csv,index=False,encoding="utf-8")
    return pop_csv,label_csv,len(lst)

# ---------- 自動判定と実行 ----------
# KML・GeoJSON を自動検出
kml_files=glob.glob("*.kml")
geo_files=glob.glob("*.geojson")
if not kml_files: raise FileNotFoundError("KMLファイルが見つかりません。")
if not geo_files: raise FileNotFoundError("GeoJSONファイルが見つかりません。")
kml=kml_files[0]; geo=geo_files[0]
print(f"使用KML: {kml}\n使用GeoJSON: {geo}")

with open(geo,"r",encoding="utf-8") as f: gj=json.load(f)
union=parse_kml_union(kml)
pts=pick_munis(union,gj,thr=0.3)
pop=load_pop()
prefix=re.sub(r"\.kml$","",kml)
pop_csv,label_csv,n=attach_and_export(pts,pop,prefix+"_重複なし")
print(f"✅ 抽出自治体数:{n}\n{pop_csv}\n{label_csv}")
files.download(pop_csv)
files.download(label_csv)


使用KML: 上越文化プラザ_iso_60min.kml
使用GeoJSON: N03-19_15_190101.geojson
✅ 抽出自治体数:11
上越文化プラザ_iso_60min_重複なし_25-44male_population.csv
上越文化プラザ_iso_60min_重複なし_25-44male_LabelMap.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# === 汎用版：KML圏域 × 国交省N03行政界 × e-Stat人口（年齢・性別制限なし：総人口） ===
!pip install -q shapely pandas openpyxl
import pandas as pd, re, json, glob, xml.etree.ElementTree as ET
from shapely.geometry import shape, Polygon
from shapely.ops import unary_union
from google.colab import files

# ---------- 共通ユーティリティ ----------
def norm_name(s):
    if pd.isna(s): return ""
    s = str(s).strip().replace("　","").replace(" ","")
    # （）内を削除
    s = re.sub(r"（.*?）|\(.*?\)", "", s)
    # ○○郡△△町 → △△町
    s = re.sub(r".+郡", "", s)
    # 「○○市△△区」→「△△区」（政令市対応）
    if "区" in s and "市" in s:
        s = re.sub(r".+市", "", s)
    return s

def parse_kml_union(path):
    ns = {"kml":"http://www.opengis.net/kml/2.2"}
    root = ET.parse(path).getroot()
    coords = root.findall(".//kml:Polygon//kml:outerBoundaryIs//kml:LinearRing//kml:coordinates", ns)
    polys=[]
    for c in coords:
        pts=[]
        for tok in re.split(r"\s+", (c.text or "").strip()):
            parts = tok.split(",")
            if len(parts)>=2:
                try:
                    pts.append((float(parts[0]), float(parts[1])))
                except:
                    pass
        if len(pts)>=3:
            p=Polygon(pts)
            if p.is_valid and p.area>0:
                polys.append(p)
    if not polys:
        raise ValueError("KMLから有効なPolygonが取得できませんでした。")
    return unary_union(polys) if len(polys)>1 else polys[0]

def pick_munis(union, gj, thr=0.3):
    rows=[]
    for ft in gj.get("features", []):
        g=ft.get("geometry")
        if not g:
            continue
        shp=shape(g)
        inter=shp.intersection(union)
        if inter.is_empty:
            continue
        ratio=inter.area/shp.area if shp.area>0 else 0
        if ratio>=thr:
            pref=ft["properties"].get("N03_001","")
            muni=(ft["properties"].get("N03_005") or
                  ft["properties"].get("N03_004") or
                  ft["properties"].get("N03_003") or
                  ft["properties"].get("N03_007") or
                  "")
            c=shp.centroid
            rows.append({"都道府県":pref,"市区町村":muni,"lat":c.y,"lng":c.x})
    return pd.DataFrame(rows).drop_duplicates(subset=["都道府県","市区町村"]).reset_index(drop=True)

# ---------- e-Stat人口（総人口化：年齢・性別制限なし） ----------
def load_pop():
    # 「年齢階級別人口」「2025.8」などを含むファイルを自動検出
    xlsx = sorted(glob.glob("*年齢階級別人口*2025.8*.xlsx"))
    if not xlsx:
        raise FileNotFoundError("人口Excelが見つかりません。ファイル名に『年齢階級別人口』『2025.8』などが含まれているか確認してください。")

    # ==== 元コードと同じ読み方（header=None で生データを読む） ====
    df_raw = pd.read_excel(
        xlsx[0],
        sheet_name="年齢別人口（市区町村別）【総計】",
        header=None,
        dtype=str
    )

    # 先頭4列 = 団体コード / 都道府県名 / 市区町村名 / 性別
    fixed = ["団体コード","都道府県名","市区町村名","性別"]

    # 1行目の 5列目以降に
    #   4列目: 「総数」
    #   5列目以降: 「0歳～4歳」「5歳～9歳」...
    # が並んでいる
    ages = df_raw.iloc[1, 4:].tolist()

    cols = fixed + ages

    # 4行目以降がデータ本体（行 index=3 以降）
    df = df_raw.iloc[3:, :len(cols)].copy()
    df.columns = cols

    # 列名を合わせる
    df = df.rename(columns={"都道府県名":"都道府県","市区町村名":"市区町村"})

    # ---- 正規化キー作成（元コードと同じ） ----
    df["都道府県_norm"] = df["都道府県"].astype(str).str.strip()
    df["市区町村_norm"] = df["市区町村"].map(norm_name)
    df["key"] = df["都道府県_norm"] + " / " + df["市区町村_norm"]

    # 市区町村合計行だけ欲しいので、「市区町村名 = '-'」等は除外
    df = df[(df["市区町村_norm"] != "") & (df["市区町村"] != "-")].copy()

    # 性別 = 「計」の行だけを使う（ここが「性別制限なし」のポイント）
    df_use = df[df["性別"].astype(str).str.strip() == "計"].copy()

    # ★ここが人口の決定ポイント★
    # 「総数」列は既に「全年齢・男女計の総人口」なので、
    # 年齢階級を足し直さず、この列だけを数値化して使う。
    df_use["Population"] = pd.to_numeric(
        df_use["総数"].astype(str).str.replace(",", ""),
        errors="coerce"
    ).fillna(0).astype(int)

    # 念のため key ごとに集計（通常は1行だけだが保険）
    pop = (
        df_use.groupby("key", as_index=False)["Population"]
              .sum()
    )

    # key（正規化済み都道府県＋市区町村）、Population（総人口）だけ返す
    return pop[["key", "Population"]]


# ---------- 人口付与 & CSV出力 ----------
def attach_and_export(df_pts, pop, prefix):
    # 重複削除
    df_pts = df_pts.drop_duplicates(subset=["都道府県","市区町村"]).reset_index(drop=True)
    df_pts["都道府県_norm"] = df_pts["都道府県"].astype(str).str.strip()
    df_pts["市区町村_norm"] = df_pts["市区町村"].map(norm_name)
    df_pts["key"] = df_pts["都道府県_norm"] + " / " + df_pts["市区町村_norm"]

    # KML圏域と人口データをkeyで結合
    res = df_pts.merge(pop, on="key", how="left")[["都道府県","市区町村","lat","lng","Population"]]

    # 同一自治体が複数featureに跨る場合：座標は平均、人口は合算
    res = (
        res.groupby(["都道府県","市区町村"], as_index=False)
           .agg({"lat":"mean","lng":"mean","Population":"sum"})
    )

    # ====== ① シンプルな人口リストCSV（日本語見出し） ======
    res["総人口"] = pd.to_numeric(res["Population"], errors="coerce").fillna(0).astype(int)
    lst = res[["都道府県","市区町村","総人口"]].sort_values(["都道府県","市区町村"]).reset_index(drop=True)

    total = pd.DataFrame({
        "都道府県": ["(合計)"],
        "市区町村": [f"対象自治体数:{len(lst)}"],
        "総人口": [lst["総人口"].sum(skipna=True)]
    })

    out = pd.concat([lst, total], ignore_index=True)

    pop_csv = f"{prefix}_population.csv"
    out.to_csv(pop_csv, index=False, encoding="utf-8-sig")

    # ====== ② My Maps 向けラベルCSV ======
    res["Population"] = res["総人口"]  # 念のためintを保証
    res["Label"] = res["市区町村"] + "（" + res["Population"].map(lambda x: f"{x:,}人") + "）"

    label_csv = f"{prefix}_LabelMap.csv"
    res.rename(columns={"市区町村":"Name"})[["Name","Label","lat","lng","Population"]].to_csv(
        label_csv,
        index=False,
        encoding="utf-8"
    )

    return pop_csv, label_csv, len(lst)

# ---------- 自動判定と実行 ----------
# KML・GeoJSON を自動検出
kml_files = glob.glob("*.kml")
geo_files = glob.glob("*.geojson")

if not kml_files:
    raise FileNotFoundError("KMLファイルが見つかりません。")
if not geo_files:
    raise FileNotFoundError("GeoJSONファイルが見つかりません。")

kml = kml_files[0]
geo = geo_files[0]
print(f"使用KML: {kml}\n使用GeoJSON: {geo}")

with open(geo, "r", encoding="utf-8") as f:
    gj = json.load(f)

union = parse_kml_union(kml)
pts   = pick_munis(union, gj, thr=0.3)
pop   = load_pop()

prefix = re.sub(r"\.kml$", "", kml)
pop_csv, label_csv, n = attach_and_export(pts, pop, prefix + "_重複なし")

print(f"✅ 抽出自治体数:{n}\n{pop_csv}\n{label_csv}")
files.download(pop_csv)
files.download(label_csv)


使用KML: イオン弘前樋の口_iso_30min.kml
使用GeoJSON: N03-20240101.geojson


KeyboardInterrupt: 